# Install Libraries:

In [ ]:
!pip install opencv-python
!pip install scenedetect


# Splitting the video

In [40]:
#detect and split the video and but the result videos in the result folder
!scenedetect -i ../videos/video.mp4  -o ../scenes detect-content split-video list-scenes save-images

[PySceneDetect] PySceneDetect v0.5.6.1
[PySceneDetect] Output directory set:
  \\192.168.1.5\2tb\Projects\Python\Style-based Video Editor\scenes
[PySceneDetect] Loaded 1 video, framerate: 29.970 FPS, resolution: 1280 x 720
[PySceneDetect] Downscale factor set to 5, effective resolution: 256 x 144
[PySceneDetect] FFmpeg codec args set: -c:v libx264 -preset veryfast -crf 22 -c:a aac
[PySceneDetect] Video output file name format: $VIDEO_NAME-Scene-$SCENE_NUMBER
[PySceneDetect] Scene list CSV file name format:
  $VIDEO_NAME-Scenes.csv
[PySceneDetect] Image output format set: JPEG [Quality: 95]
[PySceneDetect] Detecting scenes...
[PySceneDetect] Processed 4368 frames in 10.9 seconds (average 400.09 FPS).
[PySceneDetect] Detected 22 scenes, average shot length 6.6 seconds.
[PySceneDetect] Writing scene list to CSV file:
  \\192.168.1.5\2tb\Projects\Python\Style-based Video Editor\scenes\video-Scenes.csv
[PySceneDetect] Scene List:
-------------------------------------------------------------


100%|██████████| 4368/4368 [00:10<00:00, 400.46frames/s]

100%|██████████| 66/66 [00:06<00:00,  9.81images/s]

  0%|          | 0/4368 [00:00<?, ?frame/s]ffmpeg version 4.4-full_build-www.gyan.dev Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 10.2.0 (Rev6, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libdav1d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-frei0r --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf -

In [1]:
# function to get the times of each scene
from scenedetect import VideoManager, SceneManager, video_splitter


In [8]:
# check if the ffmpeg exits
video_splitter.is_ffmpeg_available()
# check if the ffmpeg exits
video_splitter.is_mkvmerge_available()

True

# Mearge Video

In [ ]:
!ffmpeg -i part1.mp4 -vcodec copy -vbsf h264_mp4toannexb -acodec copy part1.ts
!ffmpeg -i part2.mp4 -vcodec copy -vbsf h264_mp4toannexb -acodec copy part2.ts
!cat part1.ts part2.ts > parts.ts
!ffmpeg -y -i parts.ts -acodec copy -ar 44100 -ab 96k -coder ac -vbsf h264_mp4toannexb parts.mp4
!pause

# Get the times of each scene and get the frame of each scene

In [ ]:
# function to get the times of each scene
from scenedetect import VideoManager
from scenedetect import SceneManager
from scenedetect.scene_manager import save_images, write_scene_list_html
# For content-aware scene detection:
from scenedetect.detectors import ContentDetector

def find_scenes(video_path, threshold=30.0):
    # Create our video & scene managers, then add the detector.
    video_manager = VideoManager([video_path])
    scene_manager = SceneManager()
    scene_manager.add_detector(ContentDetector(threshold=threshold))

    # Improve processing speed by downscaling before processing.
    video_manager.set_downscale_factor()

    # Start the video manager and perform the scene detection.
    video_manager.start()
    scene_manager.detect_scenes(frame_source=video_manager)
    scene_list = scene_manager.get_scene_list()

    #save the images of each scene
    save_images(scene_list,video_manager,num_images=1,image_name_template='$SCENE_NUMBER',output_dir='../keyframes')
    
    # Each returned scene is a tuple of the (start, end) timecode.
    return scene_list

In [ ]:
scenes = find_scenes("../Videos/video.mp4")
scenes